In [2]:
from datasets import load_dataset
from sentence_transformers import (
    SentenceTransformer,
    SentenceTransformerTrainer,
    SentenceTransformerTrainingArguments,
    SentenceTransformerModelCardData,
)
from sentence_transformers.losses import MultipleNegativesRankingLoss
from sentence_transformers.training_args import BatchSamplers
from sentence_transformers.evaluation import InformationRetrievalEvaluator

ModuleNotFoundError: No module named 'datasets'

In [10]:
# Set parameters

batch_size = 16
epoch = 5
model_name = "all-MiniLM-L6-v2"
checkpoint_enable = False
print(f"Checkpoint: {checkpoint_enable}")
# model name : ["all-MiniLM-L6-v2", "all-mpnet-base-v2", "multi-qa-mpnet-base-dot-v1", "all-distilroberta-v1", "all-MiniLM-L12-v2", "multi-qa-distilbert-cos-v1", "multi-qa-MiniLM-L6-cos-v1"]
model_output_folder = "fine_tuned_models"
mode_output_name = f"{model_name}_fineTuned_b{batch_size}_e{epoch}_bf16"

Checkpoint: False


In [11]:
from dataset_setup import get_datasets, print_dataset_info
# 1. Load a model to finetune:
model = SentenceTransformer(model_name)

# 2. Load a dataset to finetune on
file_path = "./data/queries_corpus_300.json"
datasets = get_datasets(
    file_path, 0.9, 0
)  # Train 90% Validation 0 and Test 10% [At the end of complete training]

print_dataset_info(datasets)

Train dataset size: 8083
Validation dataset size: 0
Evaluator Val dataset size: 
	 queries: 0
	 corpus: 300
	 relevant_docs: 0
Evaluator Test dataset size: 
	 queries: 899
	 corpus: 300
	 relevant_docs: 899


In [ ]:
# 3. Define a loss function
loss = MultipleNegativesRankingLoss(model)


# 4. Specify training arguments
train_args = SentenceTransformerTrainingArguments (
    # Required parameter:
    output_dir=f"{model_output_folder}/{mode_output_name}",
    # Optional training parameters:
    num_train_epochs=epoch,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    learning_rate=2e-5,
    warmup_ratio=0.1,
    fp16=False,  # Set to False if you get an error that your GPU can't run on FP16
    bf16=False,  # Set to True if you have a GPU that supports BF16
    batch_sampler=BatchSamplers.NO_DUPLICATES,  # losses that use "in-batch negatives" benefit from no duplicates
    # Optional tracking/debugging parameters:
    evaluation_strategy="no",
    # eval_steps=100,
    save_strategy="steps",
    save_steps=50,
    save_total_limit=2,
    logging_steps=50,
    log_on_each_node=False,
    run_name=mode_output_name,  # Will be used in W&B if `wandb` is installed
)

/mnt/project_vol/.conda/envs/ashu_rag/lib/python3.13/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [13]:
# 5. Create an evaluator & evaluate the base model
eval_dataset = datasets["evaluator_test"]
dev_evaluator = InformationRetrievalEvaluator(
    eval_dataset["queries"], eval_dataset["corpus"], eval_dataset["relevant_docs"]
)

print(f"Evaluation for Base model {model_name}\n: {dev_evaluator(model)}")

Evaluation for Base model all-MiniLM-L6-v2
: {'cosine_accuracy@1': 0.7041156840934372, 'cosine_accuracy@3': 0.8954393770856507, 'cosine_accuracy@5': 0.9243604004449388, 'cosine_accuracy@10': 0.9599555061179088, 'cosine_precision@1': np.float64(0.7041156840934372), 'cosine_precision@3': np.float64(0.29847979236188354), 'cosine_precision@5': np.float64(0.18487208008898776), 'cosine_precision@10': np.float64(0.09599555061179088), 'cosine_recall@1': np.float64(0.7041156840934372), 'cosine_recall@3': np.float64(0.8954393770856507), 'cosine_recall@5': np.float64(0.9243604004449388), 'cosine_recall@10': np.float64(0.9599555061179088), 'cosine_ndcg@10': np.float64(0.8439564928809712), 'cosine_mrr@10': 0.8056398643995979, 'cosine_map@100': np.float64(0.8073769182152172)}


In [1]:
# 6. Create a trainer & train
trainer = SentenceTransformerTrainer(
    model=model,
    args=train_args,
    train_dataset=datasets["train"],
    # eval_dataset=datasets["val"], # To get validation loss details during training. May slow down trainig.
    loss=loss,
    # evaluator=dev_evaluator, # To get detailed evaluation results during trainig.
)
trainer.train(resume_from_checkpoint=checkpoint_enable)

NameError: name 'SentenceTransformerTrainer' is not defined

In [ ]:
# 7. Evaluate the trained model on the test set
dev_evaluator(model)
print(f"Evaluation for FineTuned model {model_name}\n: {dev_evaluator(model)}")

In [ ]:
# 8. Save the trained model
model.save_pretrained(
    f"{model_output_folder}/{mode_output_name}"
)